In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.interpolate import RegularGridInterpolator
import pandas as pd
import json
import sys
sys.path.append('../')
import neftpy.upvt_gas as pvtg
import neftpy.upvt_np_vect as pvtv
import scipy.optimize as opt

In [5]:
#код для конвертации данных по графику Стендинга Катца из отдельных файлов в json

tpr = [1.05, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.2, 2.4, 2.6, 2.8, 3] # имеющиеся наборы кривых
dct = {}

for t in tpr:
    data = pd.read_csv(r'data\Standing-Katz Chart Data\sk_tpr_{}.txt'.format(int(t*100)), sep=';')
    ppr = pd.DataFrame(data)['x'].to_list()
    z = pd.DataFrame(data)['y'].to_list()
    dct[t] = {'ppr':ppr, 'z':z, 'tpr':t}

with open('result.json', 'w') as fp:
    json.dump(dct, fp)



In [ ]:

def load_StandingKatz_curve():
    """
    загружает табулированные значения кривых с графика Standing Katz из набора файлов в папке data
    """
    tpr = [1.05, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.2, 2.4, 2.6, 2.8, 3] # имеющиеся наборы кривых
    df = pd.DataFrame()
    try:
        for t in tpr:
            data = pd.read_csv(r'data\Standing-Katz Chart Data\sk_tpr_{}.txt'.format(int(t*100)), sep=';')
            ppr = np.array(pd.DataFrame(data)['x'])
            z = np.array(pd.DataFrame(data)['y'])
            df_SK_t = pd.DataFrame({f"tpr = {t}":z}, index = ppr)
            if df.shape[0]==0:
                df = df_SK_t.copy()
            else:
                df =df.join(df_SK_t, how='outer')
            df = df.sort_index().interpolate(method='index')
        # пробуем сделать интерполятор на регулярной сетке
        ppr = np.array(df.index)
        tpr = np.array(tpr)
        data = np.array(df)
        interp = RegularGridInterpolator((ppr, tpr), data,
                                         bounds_error=False, fill_value=None)
        return {'df':df, 'interp':interp}
    except Exception as e:
        print(e)
